In [14]:
from glob import glob
from hfradarpy.radials import Radial
from joblib import Parallel, delayed
import multiprocessing
import os
import xarray as xr

In [8]:
rdir = "/Users/mikesmith/Documents/github/rucool/hfradarpy/examples/data/radials/ruv/SEAB"
ext = "*.ruv"

radial_list = sorted(glob(os.path.join(rdir, ext)))
radial_list[:3] # list only the first 3

['/Users/mikesmith/Documents/github/rucool/hfradarpy/examples/data/radials/ruv/SEAB/RDLi_SEAB_2019_01_01_0000.ruv',
 '/Users/mikesmith/Documents/github/rucool/hfradarpy/examples/data/radials/ruv/SEAB/RDLi_SEAB_2019_01_01_0100.ruv',
 '/Users/mikesmith/Documents/github/rucool/hfradarpy/examples/data/radials/ruv/SEAB/RDLi_SEAB_2019_01_01_0200.ruv']

In [13]:
%%timeit

radial_dict = {}
for radial in radial_list:
    if not isinstance(radial, Radial):
        radial = Radial(radial)
        radial_dict[radial.file_name] = radial.to_xarray("gridded", enhance=True)

ds = xr.concat(radial_dict.values(), "time").sortby("time")
# ds

4.67 s ± 222 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
def load_radials(radial):
    if not isinstance(radial, Radial):
        radial = Radial(radial)
        ds = radial.to_xarray("gridded", enhance=True)
    return (radial.file_name, ds)

In [17]:
%%timeit
num_cores = multiprocessing.cpu_count()
# num_cores
radials = Parallel(n_jobs=num_cores)(delayed(load_radials)(radial=r) for r in radial_list)
radial_dict = {radial: ds for (radial, ds) in radials}
ds = xr.concat(radial_dict.values(), "time").sortby("time")

821 ms ± 14.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
radials = Parallel(n_jobs=1)(delayed(load_radials)(radial=r) for r in radial_list)
radial_dict = {radial: ds for (radial, ds) in radials}
ds = xr.concat(radial_dict.values(), "time").sortby("time")

4.82 s ± 278 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
